In [32]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

# 讀入紅酒資料/了解欄位資訊
* 固定酸度(Fixed Acidity): 與葡萄酒有關的多數酸，為固定或非揮發性（不易蒸發）
* 揮發性酸度(Volatile Acidity): 葡萄酒中乙酸的含量，含量過高會導致令人不快的醋味
* 檸檬酸(Citric Acid): 少量檸檬酸可以增加葡萄酒的“清爽度”和風味
* 殘糖量(Residual Sugar): 
發酵停止後剩餘的糖量，很少發現低於1克/升的葡萄酒，而高於45克/升的葡萄酒被認為是甜的
* 氯化物(Chlorides): 酒中鹽的含量
* 游離二氧化硫(Free Sulfur Dioxide): 存在於分子SO2和亞硫酸氫根離子間平衡的游離形式，可以防止微生物生長和葡萄酒的氧化
* 總二氧化硫(Total Sulfur Dioxide): S02的自由和結合形式的數量
* 密度(Density): 水的密度(根據酒精和糖含量的百分比計算)
* 酸鹼度(pH): 描述葡萄酒的酸性或鹼性程度，從0（非常酸性）到14（非常鹼性）；大多數葡萄酒的pH值在3-4之間
* 硫酸鹽(Sulphates): 一種葡萄酒添加劑，可提高二氧化硫氣體(SO2)水平，起到抗菌和抗氧化劑的作用
* 酒精含量(Alchole): 葡萄酒的酒精含量百分比
* 品質(Quality): 輸出變量（基於感官數據，得分在0到10之間）
* (參考資料: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009)

In [33]:
data = pd.read_csv('winequality-red.csv')
rows = data.shape[0]
cols = data.shape[1] - 1
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


# 問題轉換為二元分類(品質是否高於平均)

In [34]:
idxs = (data['quality'] < np.mean(data['quality']))
data.loc[:,'label'] = 1
data.loc[idxs, 'label'] = 0
data.loc[:,['quality', 'label']]

,quality,label
0,5,0
1,5,0
2,5,0
3,6,1
4,5,0
...,...,...
1594,5,0
1595,6,1
1596,6,1
1597,5,0


# 提取出特徵和預測目標

In [35]:
X = data.iloc[:,:-2]
Y = data.iloc[:,-1]
for i in range(cols):
  X.iloc[:,i] = (X.iloc[:,i] - np.min(X.iloc[:,i])) / (np.max(X.iloc[:,i]) - np.min(X.iloc[:,i]))

In [36]:
cols = cols + 1
X['intercept'] = 1

# 設定損失函數

In [37]:
def loss(Y, Y_hat):
  cross_entropy_pos = -1 * (Y * np.log(Y_hat))
  cross_entropy_neg = -1 * ((1 - Y) * np.log(1 - Y_hat))
  cross_entropy_sum = cross_entropy_pos + cross_entropy_neg
  return(np.mean(cross_entropy_sum))

# 設定模型權重參數

In [44]:
h_num = 5
a1 = np.zeros((h_num, cols + 1))
a2 = np.zeros((h_num + 1))
print('輸入層 -> 隱藏層權重:\n')
print(a1, '\n')
print('隱藏層 -> 輸出層權重:\n')
print(a2, '\n')

輸入層 -> 隱藏層權重:

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

隱藏層 -> 輸出層權重:

[0. 0. 0. 0. 0. 0.] 



# 設定回歸模型 f(X) = a1x1 + a2x2 + ... + a11x11

In [67]:
def logistic(Z):
  O = 1 / (1 + np.exp(-Z))
  return(O)

def f(X, a1, a2):
  h_num = a1.shape[0]
  input = a1.shape[1]
  Z1 = np.zeros((rows, h_num + 1))
  Z2 = np.zeros((rows))

  ## 輸入層 -> 隱藏層

  for i in range(h_num):
    for j in range(cols):
      Z1[:,i] = Z1[:,i] + (X.iloc[:,j] * a1[i, j])
  Z1[:,-1] = 1

  ## 隱藏層 -> 輸出層

  for i in range(h_num):
    Z2 = Z2 + a2[i] * logistic(Z1[:,i])
  Y = logistic(Z2)
  return(Y)

# 開始隨機搜尋訓練

In [68]:
a1_hat = a1
a2_hat = a2
ll_hat = np.Infinity

for i in range(5000):

  ## 更新輸入層 -> 隱藏層權重

  a1_new = a1_hat + np.random.normal(0, 1, size = (h_num, cols + 1))
  Yh_new = f(X, a1_new, a2_hat)
  ll_new = loss(Y, Yh_new)
  if(ll_new < ll_hat):
    a1_hat = a1_new
    ll_hat = ll_new
    print('iteration = ', i, ', loss = ', ll_hat)

  ## 更新隱藏層 -> 輸出層權重

  a2_new = a2_hat + np.random.normal(0, 1, size = (h_num + 1))
  Yh_new = f(X, a1_hat, a2_new)
  ll_new = loss(Y, Yh_new)
  if(ll_new < ll_hat):
    a2_hat = a2_new
    ll_hat = ll_new
    print('iteration = ', i, ', loss = ', ll_hat)

iteration =  0 , loss =  0.6931471805599165
iteration =  9 , loss =  0.6906645403934227
iteration =  10 , loss =  0.6804908273296701
iteration =  11 , loss =  0.6734833692323439
iteration =  12 , loss =  0.6733128381891363
iteration =  19 , loss =  0.6709727693227207
iteration =  20 , loss =  0.670837474910417
iteration =  21 , loss =  0.655078038815637
iteration =  23 , loss =  0.6516524459928862
iteration =  24 , loss =  0.6420900379131711
iteration =  25 , loss =  0.6197055586144468
iteration =  30 , loss =  0.6113916787556729
iteration =  40 , loss =  0.6093473919363461
iteration =  51 , loss =  0.5964852333721872
iteration =  52 , loss =  0.5945159356497376
iteration =  55 , loss =  0.5792952034406154
iteration =  121 , loss =  0.5779105210462695
iteration =  139 , loss =  0.5743384491997796
iteration =  148 , loss =  0.5673343740929592
iteration =  209 , loss =  0.5656533641772493
iteration =  228 , loss =  0.5643987728170552
iteration =  231 , loss =  0.547577973194051
iteration

# 計算混淆矩陣

In [69]:
Y_hat = f(X, a1_hat, a2_hat)
Y_bin = np.array([0 for k in range(rows)]) 
Y_bin[Y_hat >= 0.5] = 1

TP = sum((Y_bin == 1) & (Y == 1))
TN = sum((Y_bin == 0) & (Y == 0))
FP = sum((Y_bin == 1) & (Y == 0))
FN = sum((Y_bin == 0) & (Y == 1))

accuracy = (TP + TN) / rows
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print('TP =', TP)
print('TN =', TN)
print('FP =', FP)
print('FN =', FN)
print('Accuracy =', np.round(accuracy, 3))
print('Precision =', np.round(precision, 3))
print('Recall =', np.round(recall, 3))

TP = 624
TN = 578
FP = 166
FN = 231
Accuracy = 0.752
Precision = 0.79
Recall = 0.73
